In [1]:
import math
import numpy as np
from qiskit import *
from qiskit import IBMQ
import matplotlib.pyplot as plt
from IPython.display import display
from qiskit import QuantumCircuit, transpile
from sympy.utilities.iterables import multiset_permutations

In [2]:
IBMQ.save_account('de404b7eba21afacf3e900bcd1cc1244967cecad8f7051683e08ff88d444aee2e6cb029f51036d2eb8efd6f18bd04acf5d9bc0891891ebe4c1bb1ab062f7741b', overwrite=True)
IBMQ.load_account()

my_provider = IBMQ.get_provider()
backend  = my_provider.get_backend('ibmq_lima')

In [3]:
x_data0 = np.loadtxt('x_data0.txt', dtype=int)
y_data = np.loadtxt('y_data0.txt', dtype=int)

max_length = 54
x_data = np.array([np.pad(row, (0, max_length-len(row))) for row in x_data0])

In [4]:
print(np.shape(x_data))
print(np.shape(x_data))

(30000, 54)
(30000, 54)


In [5]:
def get_cnot_count(qc, layout, backend):
    transpiled_circuit = transpile(qc, backend, initial_layout=layout)
    data = transpiled_circuit.count_ops()
    l = list(data.values())
    return l[0]

def get_circuit_from_x(x, backend):
    qc = QuantumCircuit(5,5)
    
    x = np.reshape(x, (-1,2))
    dimension = np.shape(x)[0]
    for i in range(dimension):
        if(x[i,0] != 0 or x[i,1] != 0):
            qc.cx(x[i,0], x[i,1])
        else:
            break
    return qc

def get_designspace(qc, backend):
    qubits = np.array([0, 1, 2, 3, 4])
    designspace = np.empty(120, dtype=int)
    i = 0
    for p in multiset_permutations(qubits):
        designspace[i] = get_cnot_count(qc, p, backend)
        i = i+1
        
    return np.array(designspace)

def get_optimal_layouts(designspace, margin, backend):
    accepted_range = np.amin(designspace) + margin
    indexes = np.where(designspace <= accepted_range)[0]
    
    return indexes

def one_hot_encode_layout_data(layouts):
    encoded = np.zeros(120, dtype=int)
    for i in layouts:
        encoded[i] = 1
        
    return encoded

def get_new_target_element(x, backend):
    datasetsize = np.shape(x)[0]
    current_cnot_data = x
    qc = get_circuit_from_x(current_cnot_data, backend)
    designspace = get_designspace(qc, backend)
    
    margin = np.std(designspace)
    
    indexes = get_optimal_layouts(designspace, margin, backend)
    target_element = one_hot_encode_layout_data(indexes)
    
    return target_element

def get_target_data(x_data, backend):
    y_new = []
    datasetsize = np.shape(x_data)[0]
    for i in range(datasetsize):
        if(np.all((x_data[i] == 0))):
            continue
        else:
            target_element = get_new_target_element(x_data[i], backend)
            y_new.append(target_element)
            
        if(i%2500 == 0):
            print("Iteration -", i)
    return y_new

In [6]:
qc = get_circuit_from_x(x_data[4852], backend)
designspace = get_designspace(qc, backend)
print("Shape -", np.shape(designspace))
print("Minimum -", np.amin(designspace))
print("Standard Deviation -", math.ceil(np.std(designspace)))
print(designspace)

Shape - (120,)
Minimum - 17
Standard Deviation - 5
[26 31 23 27 24 21 24 30 23 24 21 18 19 30 21 23 26 29 22 28 25 25 29 26
 23 28 20 26 26 23 21 33 20 21 24 18 24 28 26 26 32 30 25 22 21 23 23 19
 26 25 23 25 23 18 24 28 23 28 26 19 25 25 21 27 30 26 29 22 18 27 27 19
 17 26 20 29 26 29 20 29 23 29 27 27 18 29 17 24 22 22 23 22 21 19 29 24
 25 32 26 27 31 28 27 30 29 30 35 31 26 27 29 30 32 35 26 31 23 22 35 24]


In [7]:
indexes = get_optimal_layouts(designspace, 5, backend)
print("Indexes -\n", indexes)
s = []
for i in indexes:
    s.append(designspace[i])
s = np.array(s)
print("Design Space -\n", s)

Indexes -
 [  5  10  11  12  14  18  26  30  32  33  35  43  44  47  53  59  62  67
  68  71  72  74  78  84  86  88  89  91  92  93 117]
Design Space -
 [21 21 18 19 21 22 20 21 20 21 18 22 21 19 18 19 21 22 18 19 17 20 20 18
 17 22 22 22 21 19 22]


In [8]:
y_trial = get_new_target_element(x_data[2387], backend)
print(y_trial)

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 1 1 1 0 1 0 0 0 0 0 1 0 0 0 0 1
 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 1 0 1 0 0 0 1
 0 1 0 0 0 1 0 1 0 0 0 1 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0]


In [9]:
#y_new = get_target_data(x_data, backend)

In [10]:
non_zero_x = []
non_zero_y = []
for i in range(np.shape(x_data)[0]):
    if(~np.all((x_data[i]==0))):
        non_zero_x.append(x_data[i])
        non_zero_y.append(y_data[i])
    
non_zero_x = np.array(non_zero_x)
non_zero_y = np.array(non_zero_y)

In [11]:
print(np.array(non_zero_x))
print(np.shape(non_zero_x))

[[4 3 1 ... 0 0 0]
 [0 1 4 ... 0 0 0]
 [4 0 2 ... 0 0 0]
 ...
 [4 0 1 ... 0 0 0]
 [1 3 2 ... 0 0 0]
 [4 3 0 ... 0 0 0]]
(29850, 54)


In [12]:
print(np.array(non_zero_y))
print(np.shape(non_zero_y))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(29850, 120)


In [13]:
y_new = get_target_data(non_zero_x, backend)

Iteration - 0


IndexError: list index out of range